In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ibrahimalobaid/kitte-dataset")

print("Path to dataset files:", path)

Resuming download from 1503657984 bytes (4603804318 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/ibrahimalobaid/kitte-dataset?dataset_version_number=1 (1503657984/6107462302) bytes left.


100%|██████████| 5.69G/5.69G [04:32<00:00, 16.9MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ibrahimalobaid/kitte-dataset/versions/1


In [3]:
import os
import cv2
import shutil

# Paths
kitti_base_path = 'training'
yolo_base_path = 'data/kitti-yolo'

images_path = os.path.join(kitti_base_path, 'image_2')
labels_path = os.path.join(kitti_base_path, 'label_2')
yolo_images_path = os.path.join(yolo_base_path, 'images')
yolo_labels_path = os.path.join(yolo_base_path, 'labels')

# Make YOLO folders
os.makedirs(yolo_images_path, exist_ok=True)
os.makedirs(yolo_labels_path, exist_ok=True)

# We only keep this class
target_class = 'Car'
class_id = 0  # YOLO class index for 'Car'

# Converts KITTI bbox (xmin, ymin, xmax, ymax) to YOLO bbox (cx, cy, w, h)
def convert_bbox(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x_center = (box[0] + box[2]) / 2.0
    y_center = (box[1] + box[3]) / 2.0
    width = box[2] - box[0]
    height = box[3] - box[1]
    return (x_center * dw, y_center * dh, width * dw, height * dh)

# Process each label file
for label_file in os.listdir(labels_path):
    if not label_file.endswith('.txt'):
        continue

    image_file = label_file.replace('.txt', '.png')
    image_path = os.path.join(images_path, image_file)

    if not os.path.exists(image_path):
        print(f"⚠️ Image not found: {image_file}")
        continue

    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Could not read image: {image_file}")
        continue

    h, w, _ = image.shape

    # Read KITTI label file
    with open(os.path.join(labels_path, label_file), 'r') as lf:
        lines = lf.readlines()

    yolo_label_lines = []
    for line in lines:
        elements = line.strip().split()

        if elements[0] != target_class:
            continue  # only use 'Car' class

        # Extract and convert bounding box
        xmin = float(elements[4])
        ymin = float(elements[5])
        xmax = float(elements[6])
        ymax = float(elements[7])

        bbox = convert_bbox((w, h), (xmin, ymin, xmax, ymax))
        yolo_line = f"{class_id} {' '.join(f'{b:.6f}' for b in bbox)}\n"
        yolo_label_lines.append(yolo_line)

    # Skip if no 'Car' objects in image
    if not yolo_label_lines:
        continue

    # Write YOLO label file
    with open(os.path.join(yolo_labels_path, label_file), 'w') as yf:
        yf.writelines(yolo_label_lines)

    # Copy image to YOLO images directory
    shutil.copy(image_path, os.path.join(yolo_images_path, image_file))

print("✅ KITTI to YOLO conversion completed for class 'Car'.")


✅ KITTI to YOLO conversion completed for class 'Car'.


In [1]:
import os
import shutil
import tqdm
# === INPUT PATHS ===
input_images_base = 'UA-DETRAC_UPD_ANN/images'  # Images (JPG/PNG)
input_labels_base = 'UA-DETRAC_UPD_ANN/labels'  # YOLO labels (.txt)

# === OUTPUT PATHS ===
output_base = 'data/ua-detrac'
output_images_base = os.path.join(output_base, 'images')
output_labels_base = os.path.join(output_base, 'labels')

# === Make output directories ===
os.makedirs(output_images_base, exist_ok=True)
os.makedirs(output_labels_base, exist_ok=True)

# === Class ID for 'car' in UA-DETRAC ===
CAR_CLASS_ID = '2'

# === Process both splits: train and val ===
for split in ['train', 'val']:
    in_img_dir = os.path.join(input_images_base, split)
    in_lbl_dir = os.path.join(input_labels_base, split)
    out_img_dir = os.path.join(output_images_base, split)
    out_lbl_dir = os.path.join(output_labels_base, split)

    # Create output directories
    os.makedirs(out_img_dir, exist_ok=True)
    os.makedirs(out_lbl_dir, exist_ok=True)

    for lbl_file in os.listdir(in_lbl_dir):
        if not lbl_file.endswith('.txt'):
            continue

        input_lbl_path = os.path.join(in_lbl_dir, lbl_file)
        with open(input_lbl_path, 'r') as f:
            lines = f.readlines()

        # Filter only lines with class ID 3 (car)
        car_lines = [line for line in lines if line.strip().split()[0] == CAR_CLASS_ID]

        if not car_lines:
            continue  # Skip if no cars

        # Save filtered label
        output_lbl_path = os.path.join(out_lbl_dir, lbl_file)
        with open(output_lbl_path, 'w') as f:
            f.writelines(car_lines)

        # Copy corresponding image
        img_file = lbl_file.replace('.txt', '.jpg')  # or '.png' if your images are PNG
        input_img_path = os.path.join(in_img_dir, img_file)
        output_img_path = os.path.join(out_img_dir, img_file)

        if os.path.exists(input_img_path):
            shutil.copy(input_img_path, output_img_path)
        else:
            print(f"⚠️ Image missing for: {img_file}")

In [4]:
import os
import shutil
import random

# === Base paths ===
kitti_base = 'data/kitti-yolo'
img_dir = os.path.join(kitti_base, 'images')
lbl_dir = os.path.join(kitti_base, 'labels')

# === Output paths ===
splits = ['train', 'val']
output_dirs = {
    'train': {
        'images': os.path.join(kitti_base, 'images/train'),
        'labels': os.path.join(kitti_base, 'labels/train')
    },
    'val': {
        'images': os.path.join(kitti_base, 'images/val'),
        'labels': os.path.join(kitti_base, 'labels/val')
    }
}

# === Create output folders ===
for split in splits:
    os.makedirs(output_dirs[split]['images'], exist_ok=True)
    os.makedirs(output_dirs[split]['labels'], exist_ok=True)

# === Split logic ===
image_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
random.shuffle(image_files)

split_ratio = 0.8
split_index = int(len(image_files) * split_ratio)

split_map = {
    img: 'train' if i < split_index else 'val'
    for i, img in enumerate(image_files)
}

# === Move files ===
for img_file, split in split_map.items():
    label_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')

    src_img = os.path.join(img_dir, img_file)
    src_lbl = os.path.join(lbl_dir, label_file)

    dst_img = os.path.join(output_dirs[split]['images'], img_file)
    dst_lbl = os.path.join(output_dirs[split]['labels'], label_file)

    shutil.copy2(src_img, dst_img)
    if os.path.exists(src_lbl):
        shutil.copy2(src_lbl, dst_lbl)
    else:
        print(f"⚠️ Label not found for image {img_file}")


In [2]:
import os
import shutil

# === Source dataset paths ===
datasets = [
    'data/kitti-yolo',
    'data/ua-detrac'
]

# === Destination merged dataset ===
combined_base = 'data/combined_data'

for split in ['train', 'val']:
    combined_img_dir = os.path.join(combined_base, 'images', split)
    combined_lbl_dir = os.path.join(combined_base, 'labels', split)
    os.makedirs(combined_img_dir, exist_ok=True)
    os.makedirs(combined_lbl_dir, exist_ok=True)

    for dataset in datasets:
        src_img_dir = os.path.join(dataset, 'images', split)
        src_lbl_dir = os.path.join(dataset, 'labels', split)

        for img_file in os.listdir(src_img_dir):
            src_img_path = os.path.join(src_img_dir, img_file)
            dst_img_path = os.path.join(combined_img_dir, img_file)
            shutil.copy2(src_img_path, dst_img_path)

        for lbl_file in os.listdir(src_lbl_dir):
            src_lbl_path = os.path.join(src_lbl_dir, lbl_file)
            dst_lbl_path = os.path.join(combined_lbl_dir, lbl_file)
            shutil.copy2(src_lbl_path, dst_lbl_path)


In [9]:
import os

label_root = 'data/combined_data/labels'

for split in ['train', 'val']:
    label_dir = os.path.join(label_root, split)
    for file in os.listdir(label_dir):
        if not file.endswith('.txt'):
            continue

        path = os.path.join(label_dir, file)

        with open(path, 'r') as f:
            lines = f.readlines()

        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue  # skip malformed lines

            # force class ID to 0
            fixed_line = '0 ' + ' '.join(parts[1:]) + '\n'
            fixed_lines.append(fixed_line)

        # overwrite label file
        with open(path, 'w') as f:
            f.writelines(fixed_lines)


In [11]:
import os

def validate_yolo_dataset(dataset_path, nc=1):
    """
    dataset_path: path to combined YOLO dataset (with images/train, labels/train, etc.)
    nc: number of classes (e.g., 1 for 'car' only)
    """
    errors = []

    for split in ['train', 'val']:
        img_dir = os.path.join(dataset_path, 'images', split)
        lbl_dir = os.path.join(dataset_path, 'labels', split)

        if not os.path.isdir(img_dir) or not os.path.isdir(lbl_dir):
            errors.append(f"❌ Missing directory for split {split}")
            continue

        img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]

        for img_file in img_files:
            lbl_file = img_file.replace('.jpg', '.txt').replace('.png', '.txt')
            lbl_path = os.path.join(lbl_dir, lbl_file)

            if not os.path.exists(lbl_path):
                errors.append(f"❌ Missing label for image: {split}/{img_file}")
                continue

            with open(lbl_path, 'r') as f:
                lines = f.readlines()

            if len(lines) == 0:
                errors.append(f"⚠️ Empty label file: {split}/{lbl_file}")
                continue

            for idx, line in enumerate(lines):
                parts = line.strip().split()
                if len(parts) != 5:
                    errors.append(f"❌ Invalid line format in {split}/{lbl_file}: '{line.strip()}'")
                    continue

                try:
                    cls_id = int(parts[0])
                    if not (0 <= cls_id < nc):
                        errors.append(f"❌ Invalid class ID {cls_id} in {split}/{lbl_file}")
                    bbox = list(map(float, parts[1:]))
                    if not all(0 <= x <= 1 for x in bbox):
                        errors.append(f"❌ Bounding box out of range in {split}/{lbl_file}: {bbox}")
                except ValueError:
                    errors.append(f"❌ Non-numeric value in {split}/{lbl_file}: '{line.strip()}'")

    # Print summary
    if errors:
        print(f"🔍 Found {len(errors)} issues in dataset:\n")
        for e in errors[:50]:  # only show first 50 errors
            print(e)
        if len(errors) > 50:
            print(f"... (and {len(errors) - 50} more)")
    else:
        print("✅ Dataset looks good! No issues found.")

# === Run this ===
validate_yolo_dataset('data/combined_data', nc=1)


✅ Dataset looks good! No issues found.


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8-ghost-p2.yaml')  
model.train(
    data='data.yaml',     
    epochs=200,
    imgsz=640,
    batch=32,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    cos_lr=True,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.1,
    workers=8,
    device=1,   # Change to 'cpu' if needed
    patience=10,
    project='runs/train-ghost',
    name='yolov8-ghostp2-optimized',
    save=True,
    cache=True,
    verbose=True
)


WARNING ⚠️ no model scale passed. Assuming scale='n'.
Ultralytics 8.3.156 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:1 (NVIDIA H100 80GB HBM3 MIG 3g.40gb, 40320MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8-ghost-p2.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8-ghostp2-optimized3, nbs=64, nms=False, opset=None, optimize=False, optimizer=Ad

train: Scanning /workspace/hjs/python/data/combined_data/labels/train... 41491 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41491/41491 [00:23<00:00, 1740.68it/s]


train: New cache created: /workspace/hjs/python/data/combined_data/labels/train.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (25.1GB RAM): 100%|██████████| 41491/41491 [00:21<00:00, 1916.66it/s]


val: Fast image access ✅ (ping: 0.0±0.1 ms, read: 450.8±371.4 MB/s, size: 221.8 KB)


...

     25/200      10.4G     0.8794     0.6873      1.013         62        640: 100%|██████████| 1297/1297 [03:22<00:00,  6.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:11<00:00,  9.02it/s]


                   all       6675      13693       0.85      0.705      0.793      0.607

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     26/200      10.4G     0.8741     0.6807      1.009        107        640: 100%|██████████| 1297/1297 [05:38<00:00,  3.83it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [00:49<00:00,  2.13it/s]


                   all       6675      13693      0.852      0.704      0.793      0.608

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     27/200      10.4G       0.87     0.6776      1.007        113        640: 100%|██████████| 1297/1297 [05:39<00:00,  3.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  95%|█████████▌| 100/105 [02:05<00:08,  1.67s/it]

In [1]:
from ultralytics import YOLO
model = YOLO('runs/train-ghost/yolov8-ghostp2-optimized3/weights/last.pt')

# Resume training
model.train(
    resume=True,  # THIS is critical
    device=1
)

New https://pypi.org/project/ultralytics/8.3.158 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.156 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:1 (NVIDIA H100 80GB HBM3 MIG 3g.40gb, 40320MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=runs/train-ghost/yolov8-ghostp2-optimized3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=

train: Scanning /workspace/hjs/python/data/combined_data/labels/train.cache... 41491 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41491/41491 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (25.1GB RAM): 100%|██████████| 41491/41491 [00:19<00:00, 2101.69it/s]


val: Fast image access ✅ (ping: 0.0±0.1 ms, read: 530.8±575.3 MB/s, size: 221.8 KB)


...

                   all       6675      13693      0.853      0.723      0.804       0.62
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



11 epochs completed in 1.367 hours.
Optimizer stripped from runs/train-ghost/yolov8-ghostp2-optimized3/weights/last.pt, 3.7MB
Optimizer stripped from runs/train-ghost/yolov8-ghostp2-optimized3/weights/best.pt, 3.7MB

Validating runs/train-ghost/yolov8-ghostp2-optimized3/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:1 (NVIDIA H100 80GB HBM3 MIG 3g.40gb, 40320MiB)
YOLOv8-ghost-p2 summary (fused): 179 layers, 1,601,308 parameters, 0 gradients, 8.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 105/105 [01:19<00:00,  1.32it/s]


                   all       6675      13693      0.847      0.728      0.806      0.621
Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/train-ghost/yolov8-ghostp2-optimized3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ffc0cff05e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [6]:
from ultralytics import YOLO

model = YOLO('yolov8-ghost-p2.yaml')

model.train(
    data='data.yaml',
    evolve,
    epochs=200,
    imgsz=640,
    batch=32,
    optimizer='AdamW',
    lr0=0.0001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    cos_lr=True,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    copy_paste=0.1,
    workers=8,
    device=1,
    patience=10,
    project='runs/evolve-last',
    name='evolve-ghostp2',
    cache=True
       # This works ONLY in Python API
)


SyntaxError: positional argument follows keyword argument (2637243109.py, line 39)